## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-12-18-31-14 +0000,bbc,Trump says EU and Mexico face 30% tariff from ...,https://www.bbc.com/news/articles/cyvj13d9ylpo
1,2025-07-12-18-28-05 +0000,nypost,Possible rare mountain lion sighting in Roches...,https://nypost.com/2025/07/12/lifestyle/mounta...
2,2025-07-12-18-05-50 +0000,nyt,Trump Tariff Letters: See the Latest Rates and...,https://www.nytimes.com/2025/07/07/business/tr...
3,2025-07-12-17-23-07 +0000,nyt,Air India Plane Crash Report Says Fuel to Engi...,https://www.nytimes.com/2025/07/11/world/asia/...
4,2025-07-12-17-11-45 +0000,nypost,15-year-old illegal migrant arrested for Long ...,https://nypost.com/2025/07/12/us-news/15-year-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2305/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,13
24,air,6
25,india,6
4,tariff,5
45,nyc,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
11,2025-07-12-16-17-04 +0000,nypost,Widow of killed fire chief not satisfied with ...,https://nypost.com/2025/07/12/us-news/corey-co...,30
54,2025-07-11-19-40-14 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...,30
13,2025-07-12-15-47-59 +0000,nyt,"Trump Escalates Tariff Threats, Showing Little...",https://www.nytimes.com/2025/07/12/business/ec...,27
3,2025-07-12-17-23-07 +0000,nyt,Air India Plane Crash Report Says Fuel to Engi...,https://www.nytimes.com/2025/07/11/world/asia/...,27
29,2025-07-12-11-44-37 +0000,nyt,"As Trump Sows Tariff Confusion, Rules of Globa...",https://www.nytimes.com/2025/07/11/world/europ...,27


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
11,30,2025-07-12-16-17-04 +0000,nypost,Widow of killed fire chief not satisfied with ...,https://nypost.com/2025/07/12/us-news/corey-co...
3,27,2025-07-12-17-23-07 +0000,nyt,Air India Plane Crash Report Says Fuel to Engi...,https://www.nytimes.com/2025/07/11/world/asia/...
5,19,2025-07-12-16-58-40 +0000,nypost,College student attacked over NYC parking spot...,https://nypost.com/2025/07/12/us-news/college-...
26,18,2025-07-12-13-26-21 +0000,nypost,Brooklyn DA rules on criminal charges against ...,https://nypost.com/2025/07/12/us-news/king-of-...
54,17,2025-07-11-19-40-14 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...
25,15,2025-07-12-13-40-08 +0000,nypost,FEMA removed dozens of Camp Mystic buildings f...,https://nypost.com/2025/07/12/us-news/fema-rem...
23,14,2025-07-12-14-00-00 +0000,nypost,Christians United for Israel offers support wh...,https://nypost.com/2025/07/12/us-news/christia...
19,13,2025-07-12-14-44-37 +0000,nypost,Felix the Cat heirs claim they were cut out of...,https://nypost.com/2025/07/12/us-news/felix-th...
14,13,2025-07-12-15-29-35 +0000,nypost,NYC Councilman and Central Park 5 figure Yusuf...,https://nypost.com/2025/07/12/us-news/yusuf-sa...
1,12,2025-07-12-18-28-05 +0000,nypost,Possible rare mountain lion sighting in Roches...,https://nypost.com/2025/07/12/lifestyle/mounta...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
